In [1]:
import pickle
import numpy
import plotly
from collections import defaultdict
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

### Plotter Functions

In [2]:
def plot3D(x, y, z, xtitle, ytitle, ztitle):
    fig = go.Figure(data=[
            go.Scatter3d(x=x, 
                         y=y, 
                         z=z,
                        line=dict(
                            color='darkblue',
                            width=2
                        ) 
                    )])
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
    )

    plotly.offline.iplot(fig)
    
def processData(x,y,z):
    # Get x,y,z sorted based on y i.e TXF
    sortxyz = [(x,y,z) for y,x,z in sorted(zip(y,x,z))]
    
    x = [i[0] for i in sortxyz]
    y = [i[1] for i in sortxyz]
    z = [i[2] for i in sortxyz]
    
    return x,y,z
    
    
def plot3DWithTXF(x,y,z,xtitle, ytitle, ztitle):
    
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4']
    
    # Get x,y,z sorted based on y i.e TXF
    x,y,z = processData(x,y,z)
    
    txDictx = defaultdict(list)
    txDictz = defaultdict(list)
    
    for i in range(len(y)):
        txDictx[y[i]].append(x[i])
        txDictz[y[i]].append(z[i])

    data = []   
 
              
    cnt = 0          
    for key in txDictx.keys():
        
        x_d = txDictx[key]
        y_d = [key for i in range(len(txDictx[key]))]
        z_d = txDictz[key]

        temp = go.Scatter3d(x=x_d, 
                         y=y_d, 
                         z=z_d,
                        line=dict(
                            color=colors[cnt % len(colors)],
                            width=2
                        ) 
                    )
        cnt+=1      
        data.append(temp)
    
    fig = go.Figure(data = data)
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
    )

    plotly.offline.iplot(fig)
    
    
def plot2D(x, y, xtitle, ytitle):    
    fig = go.Figure(data=go.Scatter(x=x, y=y, mode='markers'))
    
    fig.update_layout(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
    )
    
    plotly.offline.iplot(fig)


In [6]:
filename = "logs/1500/test1/sim-summary.pickle"
input_file = open(filename,'rb')
cdp_axis, txf_axis, dai_axis = pickle.load(input_file)
input_file.close()

In [7]:
plot3DWithTXF(cdp_axis, txf_axis, dai_axis, "CDP Rate", "Transaction Fee", "DAI Price")

In [8]:
filename = "logs/mpdaizero/test2/sim-summary.pickle"
input_file = open(filename,'rb')
cdp_axis, txf_axis, dai_axis = pickle.load(input_file)
input_file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'logs/mpdaizero/test2/sim-summary.pickle'

In [ ]:
plot3DWithTXF(cdp_axis, txf_axis, dai_axis, "CDP Rate", "Transaction Fee", "DAI Price")